### US Accidents 資料分析：模型比較與時空風險預測
## 實驗設計
1. 比較有無資料前處理的影響
2. 比較有無混合採樣策略的影響
3. 使用三個模型：LightGBM, XGBoost, CatBoost（GPU加速版）
4. 包含交叉驗證和進度顯示
5. 創建時空風險預測數據供 Kepler.gl 使用

In [16]:
# ===========================
# Cell 1: 導入套件和設定
# ===========================
import os, time, json, gc, warnings
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch

# Scikit-learn
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, balanced_accuracy_score
)

# Imbalanced-learn（保留混合採樣）
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# 只保留 XGBoost
import xgboost as xgb

warnings.filterwarnings('ignore')

# GPU 檢查
print("="*60)
print("環境檢查")
print("="*60)
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print(f"XGBoost version: {xgb.__version__}")
print("="*60)


環境檢查
CUDA available: True
GPU: NVIDIA GeForce RTX 3090
GPU Memory: 23.56 GB
XGBoost version: 3.0.2


In [17]:
# ===========================
# Cell 2: 記憶體優化函數
# ===========================

def reduce_memory_usage(df, verbose=True):
    """通過改變數據類型來減少DataFrame的記憶體使用"""
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'記憶體使用減少了 {100 * (start_mem - end_mem) / start_mem:.1f}%')
        print(f'{start_mem:.2f} MB --> {end_mem:.2f} MB')
    
    return df

def clean_memory():
    """清理記憶體"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


In [18]:
# ===========================
# Cell 3: 載入資料（優化版）
# ===========================

def load_data_optimized(file_path, sample_frac=0.1):  # 使用10%資料做實驗
    """優化的資料載入"""
    print(f"\n載入資料: {file_path}")
    
    # 定義需要的欄位
    important_cols = [
        'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
        'Distance(mi)', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)',
        'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)',
        'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way',
        'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
        'Traffic_Calming', 'Traffic_Signal', 'Sunrise_Sunset', 'State'
    ]
    
    # 載入資料
    print(f"載入 {sample_frac*100}% 的資料...")
    df = pd.read_csv(file_path, usecols=lambda x: x in important_cols)
    df = df.sample(frac=sample_frac, random_state=42)
    
    print(f"載入資料大小: {df.shape}")
    print(f"記憶體使用: {df.memory_usage().sum() / 1024**2:.2f} MB")
    
    # 顯示目標變數分布
    print("\n目標變數分布:")
    severity_counts = df['Severity'].value_counts().sort_index()
    for sev, count in severity_counts.items():
        print(f"Severity {sev}: {count:,} ({count/len(df)*100:.2f}%)")
    
    return df

# 執行載入
file_path = 'us-accidents/US_Accidents_March23.csv'
df = load_data_optimized(file_path, sample_frac=0.1)


載入資料: us-accidents/US_Accidents_March23.csv
載入 10.0% 的資料...


載入資料大小: (772839, 27)
記憶體使用: 103.19 MB

目標變數分布:
Severity 1: 6,803 (0.88%)
Severity 2: 615,250 (79.61%)
Severity 3: 130,256 (16.85%)
Severity 4: 20,530 (2.66%)


In [19]:
# ===========================
# Cell 4: 基礎特徵工程函數
# ===========================

def basic_preprocessing(df):
    """基礎前處理：只處理缺失值和基本轉換"""
    df_copy = df.copy()
    
    # 處理日期時間
    df_copy['Start_Time'] = pd.to_datetime(df_copy['Start_Time'], errors='coerce')
    df_copy['End_Time'] = pd.to_datetime(df_copy['End_Time'], errors='coerce')
    
    # 計算持續時間
    df_copy['Duration_minutes'] = (df_copy['End_Time'] - df_copy['Start_Time']).dt.total_seconds() / 60
    
    # 過濾異常值
    df_copy = df_copy[(df_copy['Duration_minutes'] > 0) & (df_copy['Duration_minutes'] < 1440*7)]
    df_copy = df_copy.dropna(subset=['Start_Time'])
    
    # 提取基本時間特徵
    df_copy['Hour'] = df_copy['Start_Time'].dt.hour
    df_copy['DayOfWeek'] = df_copy['Start_Time'].dt.dayofweek
    df_copy['Month'] = df_copy['Start_Time'].dt.month
    
    # 處理缺失值（簡單填充）
    numeric_cols = df_copy.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if col != 'Severity':
            df_copy[col] = df_copy[col].fillna(df_copy[col].median())
    
    # 類別變數填充
    categorical_cols = ['Weather_Condition', 'State', 'Sunrise_Sunset']
    for col in categorical_cols:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].fillna('Unknown')
    
    # 布林型欄位轉換
    bool_cols = df_copy.select_dtypes(include=['bool']).columns
    for col in bool_cols:
        df_copy[col] = df_copy[col].astype(int)
    
    # 刪除不需要的欄位
    df_copy = df_copy.drop(['Start_Time', 'End_Time'], axis=1, errors='ignore')
    
    return df_copy

def advanced_preprocessing(df):
    """進階前處理：包含所有特徵工程"""
    df_copy = df.copy()
    
    # 先做基礎處理
    df_copy = basic_preprocessing(df_copy)
    
    # 額外的特徵工程
    # 1. 是否週末
    df_copy['IsWeekend'] = (df_copy['DayOfWeek'] >= 5).astype(int)
    
    # 2. 是否尖峰時段
    df_copy['IsRushHour'] = df_copy['Hour'].apply(
        lambda x: 1 if (6 <= x <= 9) or (16 <= x <= 19) else 0
    )
    
    # 3. 時段分類
    df_copy['TimeOfDay'] = pd.cut(df_copy['Hour'], 
                                  bins=[-1, 6, 12, 18, 24], 
                                  labels=[0, 1, 2, 3]).astype(int)
    
    # 4. 季節
    df_copy['Season'] = pd.cut(df_copy['Month'], 
                               bins=[0, 3, 6, 9, 12], 
                               labels=[0, 1, 2, 3]).astype(int)
    
    # 5. 天氣分類（如果有天氣條件）
    if 'Weather_Condition' in df_copy.columns:
        def categorize_weather(condition):
            if pd.isna(condition):
                return 0
            condition = str(condition).lower()
            if any(word in condition for word in ['clear', 'fair']):
                return 1
            elif any(word in condition for word in ['cloud', 'overcast']):
                return 2
            elif any(word in condition for word in ['rain', 'drizzle']):
                return 3
            elif any(word in condition for word in ['snow', 'sleet']):
                return 4
            elif any(word in condition for word in ['fog', 'mist']):
                return 5
            elif any(word in condition for word in ['storm', 'thunder']):
                return 6
            else:
                return 7
        
        df_copy['Weather_Category'] = df_copy['Weather_Condition'].apply(categorize_weather)
        df_copy = df_copy.drop('Weather_Condition', axis=1)
    
    # 6. 對類別變數進行標籤編碼
    label_encoders = {}
    categorical_cols = ['State', 'Sunrise_Sunset']
    for col in categorical_cols:
        if col in df_copy.columns:
            le = LabelEncoder()
            df_copy[col] = le.fit_transform(df_copy[col].astype(str))
            label_encoders[col] = le
    
    return df_copy, label_encoders

In [20]:
# ===========================
# Cell 5: 混合採樣策略
# ===========================

def apply_mixed_sampling(X_train, y_train):
    """應用混合採樣策略"""
    print("\n應用混合採樣策略...")
    
    # 計算各類別數量
    unique, counts = np.unique(y_train, return_counts=True)
    class_counts = dict(zip(unique, counts))
    print("原始分布:", class_counts)
    
    # 混合策略：對多數類欠採樣，對少數類過採樣
    median_count = int(np.median(counts))
    target_count = int(median_count * 1.5)
    
    # 第一步：欠採樣
    undersample_strategy = {}
    for cls, cnt in class_counts.items():
        if cnt > target_count:
            undersample_strategy[cls] = target_count
        else:
            undersample_strategy[cls] = cnt
    
    if len(undersample_strategy) > 0:
        rus = RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=42)
        X_temp, y_temp = rus.fit_resample(X_train, y_train)
    else:
        X_temp, y_temp = X_train, y_train
    
    # 第二步：過採樣
    temp_unique, temp_counts = np.unique(y_temp, return_counts=True)
    temp_class_counts = dict(zip(temp_unique, temp_counts))
    
    oversample_strategy = {}
    for cls, cnt in temp_class_counts.items():
        if cnt < target_count:
            oversample_strategy[cls] = target_count
        else:
            oversample_strategy[cls] = cnt
    
    if len(oversample_strategy) > 0:
        ros = RandomOverSampler(sampling_strategy=oversample_strategy, random_state=42)
        X_resampled, y_resampled = ros.fit_resample(X_temp, y_temp)
    else:
        X_resampled, y_resampled = X_temp, y_temp
    
    # 顯示新分布
    unique_new, counts_new = np.unique(y_resampled, return_counts=True)
    print("採樣後分布:", dict(zip(unique_new, counts_new)))
    
    return X_resampled, y_resampled

In [25]:
# ===========================
# Cell 6: XGBoost GPU 訓練器
# ===========================
def train_xgboost_gpu(
        X_train, X_test, y_train, y_test,
        X_val=None, y_val=None,
        *,
        objective='multi:softprob',
        num_class=4
    ):
    """
    通用 XGBoost GPU 訓練器
      - 多分類: objective='multi:softprob'，num_class=類別總數
    """
    params = {
        'objective': objective,
        # ==== 3090 GPU 最佳化 ====
        'tree_method': 'gpu_hist',
        'predictor':   'gpu_predictor',
        'gpu_id': 0,
        'max_bin': 256,
        'sampling_method': 'gradient_based',
        # ==== 常用超參 ====
        'max_depth': 8,
        'learning_rate': 0.08,
        'n_estimators': 1500,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'min_child_weight': 1,
        'gamma': 0.15,
        'lambda': 1.0,
        'alpha': 0.0,
        'n_jobs': os.cpu_count()
    }

    # 只有在多分類時才加入 num_class & eval_metric
    if objective.startswith('multi'):
        params['num_class'] = num_class
        params['eval_metric'] = ['mlogloss', 'merror']

    Model = xgb.XGBClassifier
    model = Model(**params)

    eval_set = [(X_test, y_test)]
    if X_val is not None:
        eval_set.append((X_val, y_val))

    start = time.time()
    model.fit(
        X_train, y_train,
        eval_set=eval_set,
        # early_stopping_rounds=80,
        verbose=200
    )
    train_time = time.time() - start

    preds = model.predict(X_test)
    return model, preds, train_time


In [26]:
# ===========================
# Cell 7: 交叉驗證函數
# ===========================

def cross_validate_model(model_func, X, y, cv_folds=5):
    """執行交叉驗證"""
    print(f"\n執行 {cv_folds} 折交叉驗證...")
    
    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    cv_scores = {
        'accuracy': [],
        'f1_score': [],
        'balanced_accuracy': []
    }
    
    for fold, (train_idx, val_idx) in enumerate(tqdm(skf.split(X, y), total=cv_folds, desc="CV Progress")):
        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y[train_idx], y[val_idx]
        
        # 訓練模型
        model, y_pred, _ = model_func(X_train_cv, X_val_cv, y_train_cv, y_val_cv)
        
        # 計算指標
        cv_scores['accuracy'].append(accuracy_score(y_val_cv, y_pred))
        cv_scores['f1_score'].append(f1_score(y_val_cv, y_pred, average='weighted'))
        cv_scores['balanced_accuracy'].append(balanced_accuracy_score(y_val_cv, y_pred))
        
        # 清理GPU記憶體
        clean_memory()
    
    # 計算平均值和標準差
    results = {}
    for metric, scores in cv_scores.items():
        results[f'{metric}_mean'] = np.mean(scores)
        results[f'{metric}_std'] = np.std(scores)
        print(f"{metric}: {results[f'{metric}_mean']:.4f} (+/- {results[f'{metric}_std']:.4f})")
    
    return results




In [27]:
# ===========================
# Cell 8: run_experiment（僅嚴重度分類）
# ===========================
def run_experiment(df):
    results = {}

    # ---------- Ⅰ. 嚴重度（分類） ----------
    df_cls, _ = advanced_preprocessing(df)
    df_cls = df_cls[df_cls['Severity'].isin([1,2,3,4])].dropna()

    obj_cols = df_cls.select_dtypes(include='object').columns
    df_cls[obj_cols] = df_cls[obj_cols].astype('category').apply(lambda s: s.cat.codes)

    X_cls = df_cls.drop('Severity', axis=1).values
    y_cls = df_cls['Severity'].values - 1      # 0~3

    X_tmp, X_test, y_tmp, y_test = train_test_split(
        X_cls, y_cls, test_size=0.20, random_state=42, stratify=y_cls
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_tmp, y_tmp, test_size=0.25, random_state=42, stratify=y_tmp
    )

    X_train_s, y_train_s = apply_mixed_sampling(X_train, y_train)

    print("\n--- 訓練【嚴重度】XGBoost 分類 ---")
    sev_model, sev_pred, sev_time = train_xgboost_gpu(
        X_train_s, X_test, y_train_s, y_test,
        X_val=X_val, y_val=y_val,
        objective='multi:softprob', num_class=4
    )
    sev_acc     = accuracy_score(y_test, sev_pred)
    sev_f1      = f1_score(y_test, sev_pred, average='weighted')
    sev_bal_acc = balanced_accuracy_score(y_test, sev_pred)
    print(f"Severity 分類結果 → acc: {sev_acc:.4f}, f1: {sev_f1:.4f}, bal_acc: {sev_bal_acc:.4f}, time: {sev_time:.1f}s")

    results['severity'] = {
        'acc':        float(sev_acc),
        'f1':         float(sev_f1),
        'bal_acc':    float(sev_bal_acc),
        'train_time': float(sev_time),
        'model':      sev_model
    }

    # ---------- 打印最終結果 ----------
    print("\n=== Experiment Summary ===")
    print(f"嚴重度 (分類):   acc={results['severity']['acc']:.4f}, "
          f"f1={results['severity']['f1']:.4f}, "
          f"bal_acc={results['severity']['bal_acc']:.4f}, "
          f"time={results['severity']['train_time']:.1f}s")

    return results

# 執行實驗
print("\n開始執行嚴重度分類實驗...")
results = run_experiment(df)



開始執行嚴重度分類實驗...

應用混合採樣策略...
原始分布: {0: 4082, 1: 368806, 2: 78145, 3: 12253}
採樣後分布: {0: 67798, 1: 67798, 2: 67798, 3: 67798}

--- 訓練【嚴重度】XGBoost 分類 ---
[0]	validation_0-mlogloss:1.34663	validation_0-merror:0.47987	validation_1-mlogloss:1.34655	validation_1-merror:0.47960
[200]	validation_0-mlogloss:0.72509	validation_0-merror:0.34363	validation_1-mlogloss:0.72328	validation_1-merror:0.34198
[400]	validation_0-mlogloss:0.65181	validation_0-merror:0.30183	validation_1-mlogloss:0.65098	validation_1-merror:0.29996
[600]	validation_0-mlogloss:0.60875	validation_0-merror:0.27951	validation_1-mlogloss:0.60818	validation_1-merror:0.27734
[800]	validation_0-mlogloss:0.58700	validation_0-merror:0.26829	validation_1-mlogloss:0.58691	validation_1-merror:0.26679
[1000]	validation_0-mlogloss:0.57890	validation_0-merror:0.26459	validation_1-mlogloss:0.57895	validation_1-merror:0.26292
[1200]	validation_0-mlogloss:0.57259	validation_0-merror:0.26140	validation_1-mlogloss:0.57281	validation_1-merror:0.2

In [28]:
# # ===========================
# # Cell 9: 結果視覺化
# # ===========================

# def visualize_results(results):
#     """視覺化實驗結果"""
#     # 準備數據
#     metrics = ['accuracy', 'f1_score', 'balanced_accuracy']
#     models = ['LightGBM', 'XGBoost', 'CatBoost']
#     experiments = list(results.keys())
    
#     # 創建比較圖表
#     fig, axes = plt.subplots(2, 2, figsize=(15, 12))
#     axes = axes.flatten()
    
#     for idx, metric in enumerate(metrics):
#         ax = axes[idx]
        
#         # 準備數據
#         data = []
#         for exp in experiments:
#             row = []
#             for model in models:
#                 row.append(results[exp][model][metric])
#             data.append(row)
        
#         # 繪製熱力圖
#         sns.heatmap(data, annot=True, fmt='.4f', 
#                    xticklabels=models, yticklabels=experiments,
#                    cmap='YlOrRd', ax=ax, cbar_kws={'label': metric})
#         ax.set_title(f'{metric.replace("_", " ").title()} 比較')
    
#     # 訓練時間比較
#     ax = axes[3]
#     time_data = []
#     for exp in experiments:
#         row = []
#         for model in models:
#             row.append(results[exp][model]['training_time'])
#         time_data.append(row)
    
#     sns.heatmap(time_data, annot=True, fmt='.2f', 
#                xticklabels=models, yticklabels=experiments,
#                cmap='Blues', ax=ax, cbar_kws={'label': '秒'})
#     ax.set_title('訓練時間比較 (秒)')
    
#     plt.tight_layout()
#     plt.savefig('model_comparison_results.png', dpi=300, bbox_inches='tight')
#     plt.show()
    
#     # 打印總結表格
#     print("\n" + "="*100)
#     print("實驗結果總結")
#     print("="*100)
#     print(f"{'實驗':<20} {'模型':<10} {'準確率':<10} {'F1分數':<10} {'平衡準確率':<12} {'訓練時間(秒)':<12}")
#     print("-"*100)
    
#     for exp in experiments:
#         for model in models:
#             metrics = results[exp][model]
#             print(f"{exp:<20} {model:<10} {metrics['accuracy']:<10.4f} "
#                   f"{metrics['f1_score']:<10.4f} {metrics['balanced_accuracy']:<12.4f} "
#                   f"{metrics['training_time']:<12.2f}")
    
#     # 找出最佳組合
#     best_score = 0
#     best_combo = None
#     for exp in experiments:
#         for model in models:
#             score = results[exp][model]['balanced_accuracy']
#             if score > best_score:
#                 best_score = score
#                 best_combo = (exp, model)
    
#     print(f"\n最佳組合: {best_combo[0]} - {best_combo[1]}")
#     print(f"平衡準確率: {best_score:.4f}")
    
#     return best_combo

# best_combo = visualize_results(results)


In [31]:
# ===========================
# Cell 10: Kepler.gl Data (向量化 Batch 推論，一個月預測)
# ===========================
def create_kepler_predictions_severity_batch(
    df_geo,                # 包含 Start_Lat, Start_Lng, Start_Time
    df_full_processed,     # advanced_preprocessing(df)→drop('Severity') 後的 DataFrame
    sev_model,             # 已訓練好的 XGBClassifier
    horizon_days=30,       # 只預測一個月（30 天）
    daily_times=None       # 時段列表，若為 None 則預設 [0,6,12,18]
):
    if daily_times is None:
        daily_times = [0, 6, 12, 18]

    # 1. 取得訓練用過的所有特徵欄位（除 Severity）
    feature_cols = df_full_processed.columns.tolist()
    for c in ['Hour', 'DayOfWeek', 'Month']:
        if c not in feature_cols:
            raise ValueError(f"訓練特徵裡缺少 '{c}'，請確認 advanced_preprocessing 已建立這三個欄位。")

    # 2. 計算每個特徵的平均值（作為模板）
    template_series = df_full_processed[feature_cols].mean().round(4)

    # 3. 建立熱點清單：取出經緯度四捨五入到小數點 2 位後，出現次數 ≥ 20 的前 500 個
    df_geo['lat_bin'] = df_geo['Start_Lat'].round(2)
    df_geo['lng_bin'] = df_geo['Start_Lng'].round(2)
    hot = (
        df_geo.groupby(['lat_bin','lng_bin'])
              .size()
              .reset_index(name='cnt')
    )
    hot = hot[hot['cnt'] >= 20].head(500)[['lat_bin','lng_bin']]

    # 4. 構造所有組合 (location × day × 時段)
    latest_day = df_geo['Start_Time'].max().normalize()
    rows = []
    for lat, lng in hot.values:
        for d in range(1, horizon_days + 1):
            ts_base = latest_day + pd.Timedelta(days=d)
            dow = (ts_base.weekday())  # DayOfWeek for ts_base + hr 不變
            for hr in daily_times:
                ts = ts_base + pd.Timedelta(hours=hr)
                rows.append((lat, lng, ts, hr, dow, ts.month))
    big_df = pd.DataFrame(rows, columns=['lat','lng','timestamp','Hour','DayOfWeek','Month'])

    # 5. 把模板的平均值覆蓋到 big_df，並保留時間特徵
    for col in feature_cols:
        if col in ['Hour','DayOfWeek','Month']:
            continue
        big_df[col] = float(template_series[col])

    # 6. 整理出供模型推論的特徵矩陣
    X_all = big_df[feature_cols].values  # shape = (hot_count × horizon_days × len(daily_times), len(feature_cols))

    # 7. 一次性呼叫 predict_proba
    all_probs = sev_model.predict_proba(X_all)    # shape=(n_rows, num_class)
    risk_scores = all_probs.max(axis=1)           # 取每列最大值
    # 依 risk_scores 給出 risk_level
    risk_levels = np.where(risk_scores > 0.7, 'High',
                  np.where(risk_scores > 0.4, 'Medium', 'Low'))

    # 8. 把結果填回 big_df
    big_df['risk_score'] = risk_scores
    big_df['risk_level'] = risk_levels

    return big_df

# ------------------------------
# 重新讀取地理資料，只需要 Start_Lat, Start_Lng, Start_Time
df_geo = pd.read_csv(
    file_path,
    usecols=['Start_Lat','Start_Lng','Start_Time'],
    parse_dates=['Start_Time'],
    nrows=100_000
)

# 先用 advanced_preprocessing 取得全量訓練特徵 (含 Hour, DayOfWeek, Month)
df_full_processed, _ = advanced_preprocessing(df.copy())
df_full_processed = df_full_processed.drop('Severity', axis=1)

print("\n[Kepler] 建立未來 30 天風險預測 DataFrame …")
start_time = time.time()
pred_df = create_kepler_predictions_severity_batch(
    df_geo,
    df_full_processed,
    results['severity']['model'],
    horizon_days=30,       # 只跑一個月
    daily_times=[0,6,12,18] # 每天 4 個時段
)
elapsed = time.time() - start_time
print(f"→ Batch 預測完成，共 {len(pred_df):,} 筆，花費 {elapsed:.1f} 秒")

# 直接輸出成 CSV 給 Kepler.gl
csv_out = 'accident_severity_forecast_kepler_30days.csv'
pred_df.to_csv(csv_out, index=False)
print(f"Kepler 檔案已存：{csv_out}")



[Kepler] 建立未來 30 天風險預測 DataFrame …
→ Batch 預測完成，共 60,000 筆，花費 0.6 秒
Kepler 檔案已存：accident_severity_forecast_kepler_30days.csv


In [32]:
# ===========================
# Cell 11: Kepler.gl 使用教學
# ===========================

print("\n" + "="*80)
print("Kepler.gl 使用教學")
print("="*80)

instructions = """
### 如何使用 Kepler.gl 視覺化您的數據：

1. **訪問 Kepler.gl**
   - 打開瀏覽器，訪問: https://kepler.gl/
   - 或使用 Kepler.gl 的 Jupyter 擴展（如果已安裝）

2. **上傳數據**
   - 點擊 "Add Data" 或直接拖拽 'us_accidents_kepler_data.csv' 文件到網頁上
   - 等待數據載入完成

3. **配置地圖圖層**
   建議創建以下圖層來展示不同維度的數據：

   a) **點圖層 (Point Layer) - 歷史事故**
      - Filter: type = 'historical'
      - Color: 根據 risk_score (使用紅色漸變)
      - Size: 根據 value (事故數量)
      - Radius: 5-10 pixels
      
   b) **熱力圖層 (Heatmap Layer) - 風險熱點**
      - 使用所有數據點
      - Radius: 20-30 km
      - Weight: risk_score
      - Intensity: 1-3
      
   c) **六邊形圖層 (Hexagon Layer) - 預測風險聚合**
      - Filter: type = 'prediction'
      - Height: 基於 risk_score 總和
      - Color: 使用紅黃綠漸變
      - Radius: 5-10 km

   d) **弧線圖層 (Arc Layer) - 風險傳播路徑**（可選）
      - 連接高風險區域
      - 顏色表示風險等級

4. **配置時間動畫**
   - 點擊左側面板的 "Filters"
   - 添加 timestamp 過濾器
   - 啟用時間動畫播放（點擊播放按鈕）
   - 調整播放速度和時間窗口

5. **互動功能**
   - 懸停查看詳細信息
   - 點擊數據點查看完整屬性
   - 使用圖層可見性開關比較不同視圖
   - 調整透明度查看重疊區域

6. **3D 視覺化**
   - 點擊右上角的 3D 按鈕
   - 使用滑鼠拖動旋轉視角
   - 滾輪縮放

7. **導出和分享**
   - 配置完成後，點擊 "Export Map"
   - 選擇 "Export as HTML" 生成獨立網頁
   - 或選擇 "Export Config" 保存配置文件

### 數據欄位說明：
- **lat, lng**: 地理座標
- **timestamp**: 時間戳（用於時間動畫）
- **type**: 數據類型（historical=歷史數據, prediction=預測數據）
- **value**: 歷史事故數量
- **risk_score**: 風險分數（0-1）
- **category**: 風險等級（High/Medium/Low）

### 進階技巧：
1. **多層疊加**：同時顯示歷史和預測數據，使用不同顏色區分
2. **時間比較**：使用分屏功能比較不同時期的風險分布
3. **自定義配色**：根據風險等級使用自定義顏色方案
4. **數據過濾**：結合多個過濾器查看特定條件下的風險模式
5. **地理圍欄**：繪製多邊形關注特定區域

### GPU 加速提示：
- Kepler.gl 會自動使用 WebGL 進行 GPU 加速渲染
- 如果數據量大，可以調整 "Resolution" 降低渲染精度提升性能
"""

print(instructions)

# 顯示數據預覽和統計
kepler_data = pred_df
print("\n數據預覽：")
print(kepler_data.head(10))

print(f"\n數據統計：")
print(f"- 總數據點: {len(kepler_data):,}")
print(f"- 歷史數據: {len(kepler_data[kepler_data['type'] == 'historical']):,}")
print(f"- 預測數據: {len(kepler_data[kepler_data['type'] == 'prediction']):,}")
print(f"- 唯一位置數: {kepler_data[['lat', 'lng']].drop_duplicates().shape[0]:,}")
print(f"- 高風險預測點: {len(kepler_data[(kepler_data['type'] == 'prediction') & (kepler_data['category'] == 'High')]):,}")

# 顯示GPU使用情況
if torch.cuda.is_available():
    print(f"\nGPU 記憶體使用: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU 記憶體快取: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

print("\n實驗完成！請查看生成的文件和圖表。")


Kepler.gl 使用教學

### 如何使用 Kepler.gl 視覺化您的數據：

1. **訪問 Kepler.gl**
   - 打開瀏覽器，訪問: https://kepler.gl/
   - 或使用 Kepler.gl 的 Jupyter 擴展（如果已安裝）

2. **上傳數據**
   - 點擊 "Add Data" 或直接拖拽 'us_accidents_kepler_data.csv' 文件到網頁上
   - 等待數據載入完成

3. **配置地圖圖層**
   建議創建以下圖層來展示不同維度的數據：

   a) **點圖層 (Point Layer) - 歷史事故**
      - Filter: type = 'historical'
      - Color: 根據 risk_score (使用紅色漸變)
      - Size: 根據 value (事故數量)
      - Radius: 5-10 pixels
      
   b) **熱力圖層 (Heatmap Layer) - 風險熱點**
      - 使用所有數據點
      - Radius: 20-30 km
      - Weight: risk_score
      - Intensity: 1-3
      
   c) **六邊形圖層 (Hexagon Layer) - 預測風險聚合**
      - Filter: type = 'prediction'
      - Height: 基於 risk_score 總和
      - Color: 使用紅黃綠漸變
      - Radius: 5-10 km

   d) **弧線圖層 (Arc Layer) - 風險傳播路徑**（可選）
      - 連接高風險區域
      - 顏色表示風險等級

4. **配置時間動畫**
   - 點擊左側面板的 "Filters"
   - 添加 timestamp 過濾器
   - 啟用時間動畫播放（點擊播放按鈕）
   - 調整播放速度和時間窗口

5. **互動功能**
   - 懸停查看詳細信息
   - 點擊數據點查看完整屬性
   - 使用圖層可見性開關比較不同視圖
   - 調整透明度查看重疊區域

6. **3D 視

KeyError: 'type'